# Marlowe Contract for Guessing Game

This Marlowe contract demonstrates how to store private data on the public blockchain in a Marlowe contract. It uses the fact that it is difficult to factor the product of two prime numbers.

⚠️ *NB: This is just an illustrative example to stretech the limits of the Marlowe language. It is only lightly tested and has not undergone any cryptographic review.*


## Rules of the Guessing Game

The gist of the game is that Bob needs to guess whether Alice secretly selected a positive or a negative number. If Bob chooses correctly, they win a wager; otherwise, Alice wins the wager.

1.   Alice and Bob each deposit the wager.
2.   Alice secretly selects two 24-digit prime numbers from https://bigprimes.org/. Call these `Smaller Secret Prime` and `Larger Secret Prime`.
3.   Alice makes the contract choice `Product of Secret Primes` with the product of `Smaller Secret Prime` and `Larger Secret Prime`.
4.   Alices secretly selects a random 23-digit positive or negative number. Call this `Plain Text`.
5.   Alices adds `Plain Text` to `Smaller Secret Prime` and makes the contract choice `Cipher Text`.
6.   Bob makes the contract choice `1` if they bet that Alice's `Plain Text` is a positive number. Otherwise Bob makes the contract choice `0`.
7.   Alice reveals their `Smaller Secret Prime` by making that contract choice.
8.   If Alice's `Smaller Secret Prime` is not a smaller factor of `Product of Secret Primes`, then Alice loses her wager.
9.   Otherwise, Bob may challenge Alice's `Smaller Secret Prime` by making the contract choice `Smaller Challenge Prime`. If that is indeed a smaller prime factor, then they win the wager; otherwise, Alice wins the wager. Bob can use a Linux utility like `factor` to find if Alice cheated by not providing the smallest prime factor of the `Product of Secret Primes`.
10.   Lastly, the contract computes `Plain Text` and awards the wager to Bob if they correctly guessed that the `Plain Text` was positive. Otherwise, the contract awards the wager to Alice.


## Choosing the Secret Primes

*   The number of digits in the product of the two primes has to be large enough so that Bob *cannot* factor the `Product of Secret Primes` for less money than is the wager is worth.
*   However, the number of digits in the product of primes has to be small enough so that Bob *can* cheaply factor the `Product of Secret Primes` if it is the product of three or more primes instead of just two.
*   A 48-digit `Product of Secret Primes` seems appropriate because it takes a long time for the Linux `factor` program to factor the product of two 24-digit primes, but a short time for it to factor the product of three 16-digit primes.


## Examples

### Alice Doesn't Cheat

1.  Alice and Bob make their deposits.
2.  Alice secretly selects the primes 418133219637912482167661 and 442045966004935111389041
3.  Alice chooses `Product of Secret Primes` = 184834102993594727417632924514462037620360003101.
4.  Alice secretly selects `Plain Text` = 19295759350583799993723
5.  Alice chooses `Cipher Text` = 437428978988496282161384.
6.  Bob chooses `Plain Text is Positive` = 0, guessing that Alice's plain text is less than zero.
7.  Alice chooses `Smaller Secret Prime` = 418133219637912482167661.
8.  Bob runs the Linux command `factor 184834102993594727417632924514462037620360003101`, but it doesn't find any factors in many minutes.
9.  Bob lets the next timeout pass, since they do not want to challenge Alice.
10. The contract pays the wager to Alice because Bob incorrectly guessed correctly that `Plain Text` was negative.

### Alice Does Cheat

1.  Alice and Bob make their deposits.
2.  Alice secretly selects three primes 6240425729022497, 7445375476229597, and 9149696769535051.
3.  Alices chooses `Product of Secret Primes` = 425116072270803373983726731189192374944067744159.
4.  Alice cleverly chooses `Cipher Text` = 6984737363688812, which is between the two lower primes.
5.  Bob chooses `Plain Text is Positive`= 1, guessing that Alice's plain text is greater than zero.
6.  Since Alice sees that Bob guessed negative, Alice chooses `Smaller Secret Prime` = 7445375476229597, even though that is false, since this would mean that `Plain Text` = `Cipher Text` - `Smaller Secret Prime` would be negative and Bob would lose the wager.
7.  However, Bob runs the Linux command `factor 190125153026841891465807101762005016303854862683`, and in fifteen seconds discovers the three factors and sees that 6240425729022497 is lower than the one that Alice reported.
8.  Bob chooses `Smaller Challenge Prime` = 6240425729022497.
9.  The contract pays Bob because Bob proved that Alice cheated. If Bob had not challenged Alice, Alice's cheating would not have been detected and Alice would have won the wager.

## Preliminaries

Make a note of the versions of the Marlowe repository and software.

In [1]:
marlowe-cli --version

marlowe-cli 0.0.5.0


In [2]:
cardano-cli --version

cardano-cli 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

de3ca53f994f989ba7f5693e5e03bdfbcd52beee


### Select the Cardano network

In [4]:
export CARDANO_TESTNET_MAGIC=1567
export CARDANO_NODE_SOCKET_PATH=node.socket

### Set up keys, addresses, and tokens

This section can be omitted if only off-chain simulation is performed.

We use public key hashes instead of role tokens to identify the participants in the contract.

### Parties to the Contract

#### Alice

In [5]:
ALICE_SKEY=alice.skey
ALICE_VKEY=alice.vkey
ALICE_PKH=$(cardano-cli address key-hash --payment-verification-key-file $ALICE_VKEY)
ALICE_ADDR=$(cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file $ALICE_VKEY)
echo $ALICE_ADDR
echo $ALICE_PKH

addr_test1vryrafsnj3wt3as5pgeng8ddh42gq0dk8gphkz3mx8utzncet8j7q
c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f


Alice has these UTxOs at her address:

In [6]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ALICE_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
96ca02355fd1fea54ee5865695bc88c94b301c614a714f8bf7c78a92db1891e8     1        202000000 lovelace + TxOutDatumNone
be211601347db446ec179e6659fa9a7c9d01f10d8912fa424179d7936dd9fb55     0        891717640 lovelace + TxOutDatumNone


Make a note of this UTxO for later use.

In [7]:
TX_0_ALICE=$(marlowe-cli util select --lovelace-only 300000000 $ALICE_ADDR | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/')
echo $TX_0_ALICE

be211601347db446ec179e6659fa9a7c9d01f10d8912fa424179d7936dd9fb55#0


#### Bob

In [8]:
BOB_SKEY=bob.skey
BOB_VKEY=bob.vkey
BOB_PKH=$(cardano-cli address key-hash --payment-verification-key-file $BOB_VKEY)
BOB_ADDR=$(cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file $BOB_VKEY)
echo $BOB_ADDR
echo $BOB_PKH

addr_test1vzl43spe69knxgfl5eqxrr89lwkef3elskmapjvzmy6akmc29ya5n
bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f


Bob has these UTxOs at his address:

In [9]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $BOB_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
96ca02355fd1fea54ee5865695bc88c94b301c614a714f8bf7c78a92db1891e8     0        895799596 lovelace + TxOutDatumNone


Make a note of these UTxOs for later use.

In [10]:
TX_0_BOB=$(marlowe-cli util select --lovelace-only 300000000 $BOB_ADDR | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/')
echo $TX_0_BOB

96ca02355fd1fea54ee5865695bc88c94b301c614a714f8bf7c78a92db1891e8#0


### Set up the contract

Start the contract at the current time.

In [11]:
NOW=$(($(date -u +%s) * 1000))
echo $NOW

1658677424000


The game lasts 30 minutes.

In [12]:
MINUTES=$((60 * 1000))
END=$((NOW + 30 * MINUTES))
echo $END

1658679224000


All of the transactions for the game will be sent before the 30 minutes.

In [13]:
LATER=$((END - 1000))
echo $LATER

1658679223000


There is a minimum Ada requirement associated with a native token.

In [14]:
ADA=1000000
MIN_ADA=$((2 * ADA))
echo $MIN_ADA

2000000


Create a Haskell program that will generate a contract with six rounds of bidding for three bidders. We could use `runhaskell` instead of `ghc`, but `runhaskell` is slow because it doesn't perform optimizations.

In [15]:
WAGER=$((100 * ADA))
echo $WAGER

100000000


In [16]:
runhaskell << EOI

{-# LANGUAGE OverloadedStrings #-}

module GuessingGame where

import Data.Aeson (encodeFile)
import Language.Marlowe.Core.V1.Semantics.Types
import Plutus.V1.Ledger.Api (POSIXTime(..))

main :: IO ()
main =
  encodeFile "guessing-game-01.contract"
    $ When
    [Case
        (Deposit
            (PK "$ALICE_PKH")
            (PK "$ALICE_PKH")
            (Token "" "")
            (Constant $WAGER)
        )
        (When
            [Case
                (Deposit
                    (PK "$BOB_PKH")
                    (PK "$BOB_PKH")
                    (Token "" "")
                    (Constant $WAGER)
                )
                (When
                    [Case
                        (Choice
                            (ChoiceId
                                "Product of Two Primes"
                                (PK "$ALICE_PKH")
                            )
                            [Bound 10000000000000000000000000000000000000000000000 100000000000000000000000000000000000000000000000000]
                        )
                        (When
                            [Case
                                (Choice
                                    (ChoiceId
                                        "Cipher Text"
                                        (PK "$ALICE_PKH")
                                    )
                                    [Bound 1000000000000000 100000000000000000000000000]
                                )
                                (When
                                    [Case
                                        (Choice
                                            (ChoiceId
                                                "Plain Text is Positive"
                                                (PK "$BOB_PKH")
                                            )
                                            [Bound 0 1]
                                        )
                                        (When
                                            [Case
                                                (Choice
                                                    (ChoiceId
                                                        "Smaller Secret Prime"
                                                        (PK "$ALICE_PKH")
                                                    )
                                                    [Bound 1000000000000000 10000000000000000000000000]
                                                )
                                                (Let
                                                    "Larger Secret Prime"
                                                    (DivValue
                                                        (ChoiceValue
                                                            (ChoiceId
                                                                "Product of Two Primes"
                                                                (PK "$ALICE_PKH")
                                                            ))
                                                        (ChoiceValue
                                                            (ChoiceId
                                                                "Smaller Secret Prime"
                                                                (PK "$ALICE_PKH")
                                                            ))
                                                    )
                                                    (If
                                                        (AndObs
                                                            (ValueLT
                                                                (ChoiceValue
                                                                    (ChoiceId
                                                                        "Smaller Secret Prime"
                                                                        (PK "$ALICE_PKH")
                                                                    ))
                                                                (UseValue "Larger Secret Prime")
                                                            )
                                                            (ValueEQ
                                                                (ChoiceValue
                                                                    (ChoiceId
                                                                        "Product of Two Primes"
                                                                        (PK "$ALICE_PKH")
                                                                    ))
                                                                (MulValue
                                                                    (ChoiceValue
                                                                        (ChoiceId
                                                                            "Smaller Secret Prime"
                                                                            (PK "$ALICE_PKH")
                                                                        ))
                                                                    (UseValue "Larger Secret Prime")
                                                                )
                                                            )
                                                        )
                                                        (When
                                                            [Case
                                                                (Choice
                                                                    (ChoiceId
                                                                        "Smaller Challenge Prime"
                                                                        (PK "$BOB_PKH")
                                                                    )
                                                                    [Bound 2 100000000000000000000000000000000000000000000000000]
                                                                )
                                                                (Let
                                                                    "Larger Challenge Prime"
                                                                    (DivValue
                                                                        (ChoiceValue
                                                                            (ChoiceId
                                                                                "Product of Two Primes"
                                                                                (PK "$ALICE_PKH")
                                                                            ))
                                                                        (ChoiceValue
                                                                            (ChoiceId
                                                                                "Smaller Challenge Prime"
                                                                                (PK "$BOB_PKH")
                                                                            ))
                                                                    )
                                                                    (If
                                                                        (AndObs
                                                                            (ValueLT
                                                                                (ChoiceValue
                                                                                    (ChoiceId
                                                                                        "Smaller Challenge Prime"
                                                                                        (PK "$BOB_PKH")
                                                                                    ))
                                                                                (ChoiceValue
                                                                                    (ChoiceId
                                                                                        "Smaller Secret Prime"
                                                                                        (PK "$ALICE_PKH")
                                                                                    ))
                                                                            )
                                                                            (ValueEQ
                                                                                (ChoiceValue
                                                                                    (ChoiceId
                                                                                        "Product of Two Primes"
                                                                                        (PK "$ALICE_PKH")
                                                                                    ))
                                                                                (MulValue
                                                                                    (ChoiceValue
                                                                                        (ChoiceId
                                                                                            "Smaller Challenge Prime"
                                                                                            (PK "$BOB_PKH")
                                                                                        ))
                                                                                    (UseValue "Larger Challenge Prime")
                                                                                )
                                                                            )
                                                                        )
                                                                        (Pay
                                                                            (PK "$ALICE_PKH")
                                                                            (Account (PK "$BOB_PKH"))
                                                                            (Token "" "")
                                                                            (Constant $WAGER)
                                                                            Close 
                                                                        )
                                                                        (Pay
                                                                            (PK "$BOB_PKH")
                                                                            (Account (PK "$ALICE_PKH"))
                                                                            (Token "" "")
                                                                            (Constant $WAGER)
                                                                            Close 
                                                                        )
                                                                    )
                                                                )]
                                                            (POSIXTime $END)
                                                            (Let
                                                                "Plain Text"
                                                                (SubValue
                                                                    (ChoiceValue
                                                                        (ChoiceId
                                                                            "Cipher Text"
                                                                            (PK "$ALICE_PKH")
                                                                        ))
                                                                    (ChoiceValue
                                                                        (ChoiceId
                                                                            "Smaller Secret Prime"
                                                                            (PK "$ALICE_PKH")
                                                                        ))
                                                                )
                                                                (If
                                                                    (ValueGT
                                                                        (MulValue
                                                                            (SubValue
                                                                                (MulValue
                                                                                    (Constant 2)
                                                                                    (ChoiceValue
                                                                                        (ChoiceId
                                                                                            "Plain Text is Positive"
                                                                                            (PK "$BOB_PKH")
                                                                                        ))
                                                                                )
                                                                                (Constant 1)
                                                                            )
                                                                            (UseValue "Plain Text")
                                                                        )
                                                                        (Constant 0)
                                                                    )
                                                                    (Pay
                                                                        (PK "$ALICE_PKH")
                                                                        (Account (PK "$BOB_PKH"))
                                                                        (Token "" "")
                                                                        (Constant $WAGER)
                                                                        Close 
                                                                    )
                                                                    (Pay
                                                                        (PK "$BOB_PKH")
                                                                        (Account (PK "$ALICE_PKH"))
                                                                        (Token "" "")
                                                                        (Constant $WAGER)
                                                                        Close 
                                                                    )
                                                                )
                                                            )
                                                        )
                                                        (Pay
                                                            (PK "$ALICE_PKH")
                                                            (Account (PK "$BOB_PKH"))
                                                            (Token "" "")
                                                            (Constant $WAGER)
                                                            Close 
                                                        )
                                                    )
                                                )]
                                            (POSIXTime $END)
                                            Close 
                                        )]
                                    (POSIXTime $END)
                                    Close 
                                )]
                            (POSIXTime $END)
                            Close 
                        )]
                    (POSIXTime $END)
                    Close 
                )]
            (POSIXTime $END)
            Close 
        )]
    (POSIXTime $END)
    Close 
    
EOI

### Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [17]:
yaml2json << EOI > guessing-game-01.state
accounts:
- - - pk_hash: $ALICE_PKH
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [18]:
json_pp < guessing-game-01.state

{
   "accounts" : [
      [
         [
            {
               "pk_hash" : "c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f"
            },
            {
               "currency_symbol" : "",
               "token_name" : ""
            }
         ],
         2000000
      ]
   ],
   "boundValues" : [],
   "choices" : [],
   "minTime" : 1658677424000
}


## Examples

### Alice Doesn't Cheat

1.  Alice and Bob make their deposits.
2.  Alice secretly selects the primes 418133219637912482167661 and 442045966004935111389041
3.  Alice chooses `Product of Secret Primes` = 184834102993594727417632924514462037620360003101.
4.  Alice secretly selects `Plain Text` = 19295759350583799993723
5.  Alice chooses `Cipher Text` = 437428978988496282161384.
6.  Bob chooses `Plain Text is Positive` = 0, guessing that Alice's plain text is less than zero.
7.  Alice chooses `Smaller Secret Prime` = 418133219637912482167661.
8.  Bob runs the Linux command `factor 184834102993594727417632924514462037620360003101`, but it doesn't find any factors in many minutes.
9.  Bob lets the next timeout pass, since they do not want to challenge Alice.
10. The contract pays the wager to Alice because Bob incorrectly guessed correctly that `Plain Text` was negative.

#### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract.

In [19]:
marlowe-cli run initialize --contract-file guessing-game-01.contract  \
                           --state-file    guessing-game-01.state     \
                           --out-file      guessing-game-01a.marlowe  \
                           --print-stats


Validator size: 12582
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}


The minimum Ada deposit has been recorded.

In [20]:
jq .state guessing-game-01a.marlowe | json2yaml

accounts:
- - - pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1658677424000


Find the script address

In [21]:
SCRIPT_ADDR=$(jq -r .marloweValidator.address guessing-game-01a.marlowe)
echo $SCRIPT_ADDR

addr_test1wp3v2mx0ccsh4l6kjtsa86lgnsss20f3lsgcst9jr07axpcmajgcl


Submit the transaction.

In [22]:
TX_1=$(
marlowe-cli run execute --tx-in "$TX_0_ALICE"                        \
                        --required-signer $ALICE_SKEY                \
                        --change-address $ALICE_ADDR                 \
                        --marlowe-out-file guessing-game-01a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_1\""


Fee: Lovelace 286237
Size: 2731 / 32768 = 8%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "55a7f12d0032651bf7014e024fde60b2a8ab73dd3bcf1f36b67375b675e3fe05"


The Marlowe script has this UTxO at its address:

In [23]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
55a7f12d0032651bf7014e024fde60b2a8ab73dd3bcf1f36b67375b675e3fe05     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "6649f9eb200ba725d384a8de5c31fc46be03f2b6309b559effa7e9fc785a3704"


Alice has these UTxOs at her address:

In [24]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ALICE_ADDR | sed -n -e "1,2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
55a7f12d0032651bf7014e024fde60b2a8ab73dd3bcf1f36b67375b675e3fe05     0        889431403 lovelace + TxOutDatumNone


#### 2. Alice makes her deposit

In [25]:
marlowe-cli run prepare --deposit-account PK=$ALICE_PKH          \
                        --deposit-party   PK=$ALICE_PKH          \
                        --deposit-amount $WAGER                  \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-01a.marlowe \
                        --out-file     guessing-game-02a.marlowe \
                        --print-stats


Datum size: 2391


Now the contract still contains the initial Ada and the deposit.

In [26]:
jq .state guessing-game-02a.marlowe | json2yaml

accounts:
- - - pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
    - currency_symbol: ''
      token_name: ''
  - 102000000
boundValues: []
choices: []
minTime: 1658677424000


Submit the transaction.

In [27]:
TX_2=$(
marlowe-cli run execute --marlowe-in-file guessing-game-01a.marlowe  \
                        --tx-in-marlowe "$TX_1#1"                    \
                        --tx-in-collateral "$TX_1#0"                 \
                        --tx-in "$TX_1#0"                            \
                        --required-signer $ALICE_SKEY                \
                        --change-address $ALICE_ADDR                 \
                        --marlowe-out-file guessing-game-02a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_2\""


Fee: Lovelace 1531805
Size: 17896 / 32768 = 54%
Execution units:
  Memory: 6661076 / 30000000 = 22%
  Steps: 2627344382 / 10000000000 = 26%
TxId "86deba4e8449ddf1ac0ebcf2d26f04a50925a198c5d95b1dac34911098f78b3d"


The Marlowe script has this UTxO at its address:

In [28]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
86deba4e8449ddf1ac0ebcf2d26f04a50925a198c5d95b1dac34911098f78b3d     1        102000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "62b8fd07313ae75ed140022066412a12929769046278f1803b022c52ca5db7cf"


Alice has these UTxOs at her address:

In [29]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ALICE_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
86deba4e8449ddf1ac0ebcf2d26f04a50925a198c5d95b1dac34911098f78b3d     0        787899598 lovelace + TxOutDatumNone


#### 3. Bob makes his deposit

In [30]:
marlowe-cli run prepare --deposit-account PK=$BOB_PKH            \
                        --deposit-party   PK=$BOB_PKH            \
                        --deposit-amount $WAGER                  \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-02a.marlowe \
                        --out-file     guessing-game-03a.marlowe \
                        --print-stats


Datum size: 2331


The contract's internal accounts have been updated.

In [31]:
jq .state.accounts guessing-game-03a.marlowe | json2yaml

- - - pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
    - currency_symbol: ''
      token_name: ''
  - 102000000
- - - pk_hash: bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f
    - currency_symbol: ''
      token_name: ''
  - 100000000


Submit the transaction.

In [32]:
TX_3=$(
marlowe-cli run execute --marlowe-in-file guessing-game-02a.marlowe  \
                        --tx-in-marlowe "$TX_2#1"                    \
                        --tx-in-collateral "$TX_0_BOB"               \
                        --tx-in "$TX_0_BOB"                          \
                        --required-signer $BOB_SKEY                  \
                        --change-address $BOB_ADDR                   \
                        --marlowe-out-file guessing-game-03a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_3\""


Fee: Lovelace 1505771
Size: 17727 / 32768 = 54%
Execution units:
  Memory: 6451112 / 30000000 = 21%
  Steps: 2537418722 / 10000000000 = 25%
TxId "c32b3e394f0c5e9d02afd6d7695bf8e22af1511574c8bc48046b4a53688ce2ef"


The Marlowe script has this UTxO at its address:

In [33]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c32b3e394f0c5e9d02afd6d7695bf8e22af1511574c8bc48046b4a53688ce2ef     1        202000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "2c064c4069bc5af1da89673cccd42d8b5f809d07bd8c31f2bcf744fa12100d10"


Bob has these UTxOs at his address:

In [34]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $BOB_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c32b3e394f0c5e9d02afd6d7695bf8e22af1511574c8bc48046b4a53688ce2ef     0        794293825 lovelace + TxOutDatumNone


#### 4. Alice secretly selects prime numbers and chooses their product

Alice visits https://bigprimes.org/ and secretly selects the primes 418133219637912482167661 and 442045966004935111389041.

In [35]:
SMALLER_PRIME=418133219637912482167661
LARGER_PRIME=442045966004935111389041
PRODUCT_OF_PRIMES=$(echo "$SMALLER_PRIME * $LARGER_PRIME" | bc)
echo $PRODUCT_OF_PRIMES

184834102993594727417632924514462037620360003101


In [36]:
marlowe-cli run prepare --choice-party PK=$ALICE_PKH             \
                        --choice-name "Product of Two Primes"    \
                        --choice-number $PRODUCT_OF_PRIMES       \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-03a.marlowe \
                        --out-file     guessing-game-04a.marlowe \
                        --print-stats


Datum size: 2276


The choice has been recorded.

In [37]:
jq .state.choices guessing-game-04a.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 1.8483410299359473e+47


👉 Note that `jq` rounds off the number when printing it, even though the full precision is present in the file `guessing-game-04a.marlowe`.

Submit the transaction.

In [38]:
TX_4=$(
marlowe-cli run execute --marlowe-in-file guessing-game-03a.marlowe  \
                        --tx-in-marlowe "$TX_3#1"                    \
                        --tx-in-collateral "$TX_2#0"                 \
                        --tx-in "$TX_2#0"                            \
                        --required-signer $ALICE_SKEY                \
                        --change-address $ALICE_ADDR                 \
                        --marlowe-out-file guessing-game-04a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_4\""


Fee: Lovelace 1496998
Size: 17615 / 32768 = 53%
Execution units:
  Memory: 6424498 / 30000000 = 21%
  Steps: 2505395063 / 10000000000 = 25%
TxId "db606e11c373e94669a43359b43a59e76a354836661f6e957e5cedc1a9421cbc"


The Marlowe script has this UTxO at its address:

In [39]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
db606e11c373e94669a43359b43a59e76a354836661f6e957e5cedc1a9421cbc     1        202000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "a7581abfb59793a27e5004ee31fcf663ecb6a59a4df1da7a472131379475e150"


Alice has these UTxOs at her address:

In [40]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ALICE_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
db606e11c373e94669a43359b43a59e76a354836661f6e957e5cedc1a9421cbc     0        786402600 lovelace + TxOutDatumNone


#### 5. Alice chooses the cipher text

Alice secretly selects Plain Text = 19295759350583799993723, which is a positive number.

In [41]:
PLAIN_TEXT=19295759350583799993723

The cipher text is the sum of the plain text and the smaller prime number.

In [42]:
CIPHER_TEXT=$(echo "$SMALLER_PRIME + $PLAIN_TEXT" | bc)
echo $CIPHER_TEXT

437428978988496282161384


In [43]:
marlowe-cli run prepare --choice-party PK=$ALICE_PKH             \
                        --choice-name "Cipher Text"              \
                        --choice-number $CIPHER_TEXT             \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-04a.marlowe \
                        --out-file     guessing-game-05a.marlowe \
                        --print-stats


Datum size: 2234


The choice has been recorded.

In [44]:
jq .state.choices guessing-game-05a.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 1.8483410299359473e+47
- - choice_name: Cipher Text
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 437428978988496300000000


Submit the transaction.

In [45]:
TX_5=$(
marlowe-cli run execute --marlowe-in-file guessing-game-04a.marlowe  \
                        --tx-in-marlowe "$TX_4#1"                    \
                        --tx-in-collateral "$TX_4#0"                 \
                        --tx-in "$TX_4#0"                            \
                        --required-signer $ALICE_SKEY                \
                        --change-address $ALICE_ADDR                 \
                        --marlowe-out-file guessing-game-05a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_5\""


Fee: Lovelace 1487714
Size: 17498 / 32768 = 53%
Execution units:
  Memory: 6384950 / 30000000 = 21%
  Steps: 2479684064 / 10000000000 = 24%
TxId "c9bb2af53136b4b172aa0e0a6a8642aec8c96c35ba97952819b04d7104cfd2c7"


The Marlowe script has this UTxO at its address:

In [46]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c9bb2af53136b4b172aa0e0a6a8642aec8c96c35ba97952819b04d7104cfd2c7     1        202000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "12af479170b3f15f2c2124c48d240791e0eafd9c0f1a1ab2d98547cba0ae1321"


Alice has these UTxOs at her address:

In [47]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ALICE_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c9bb2af53136b4b172aa0e0a6a8642aec8c96c35ba97952819b04d7104cfd2c7     0        784914886 lovelace + TxOutDatumNone


#### 6. Bob guesses that the plain text is a negative number

Before guessing whether the plain text is a positive number or not, Bob tries to discover Alice's secret primes by trying to factor the large product that Alice reported.

In [48]:
timeout 10m factor $PRODUCT_OF_PRIMES

: 124

Bob couldn't factor the product, so he just guesses that Alice's plain text is a negative number.

In [49]:
marlowe-cli run prepare --choice-party PK=$BOB_PKH               \
                        --choice-name "Plain Text is Positive"   \
                        --choice-number 0                        \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-05a.marlowe \
                        --out-file     guessing-game-06a.marlowe \
                        --print-stats


Datum size: 2201


The choice has been recorded.

In [50]:
jq .state.choices guessing-game-06a.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 1.8483410299359473e+47
- - choice_name: Cipher Text
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 437428978988496300000000
- - choice_name: Plain Text is Positive
    choice_owner:
      pk_hash: bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f
  - 0


Submit the transaction.

In [51]:
TX_6=$(
marlowe-cli run execute --marlowe-in-file guessing-game-05a.marlowe  \
                        --tx-in-marlowe "$TX_5#1"                    \
                        --tx-in-collateral "$TX_3#0"                 \
                        --tx-in "$TX_3#0"                            \
                        --required-signer $BOB_SKEY                  \
                        --change-address $BOB_ADDR                   \
                        --marlowe-out-file guessing-game-06a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_6\""


Fee: Lovelace 1474847
Size: 17423 / 32768 = 53%
Execution units:
  Memory: 6279362 / 30000000 = 20%
  Steps: 2431493547 / 10000000000 = 24%
TxId "77eb1ca8aa467a8d649ebffcaf669b09c9a58593b4ac79e6ae5d420b7e5fe917"


The Marlowe script has this UTxO at its address:

In [52]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p;/$TX_6/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
77eb1ca8aa467a8d649ebffcaf669b09c9a58593b4ac79e6ae5d420b7e5fe917     1        202000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "817bb9f886167e83bb63709b934e74bb3a807c8eee7597d7f8c10b6a4bd5ff3f"


Bob has these UTxOs at his address:

In [53]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $BOB_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p;/$TX_6/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
77eb1ca8aa467a8d649ebffcaf669b09c9a58593b4ac79e6ae5d420b7e5fe917     0        792818978 lovelace + TxOutDatumNone


#### 7. Alice reveals the smallest prime factor

In [54]:
marlowe-cli run prepare --choice-party PK=$ALICE_PKH             \
                        --choice-name "Smaller Secret Prime"     \
                        --choice-number $SMALLER_PRIME           \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-06a.marlowe \
                        --out-file     guessing-game-07a.marlowe \
                        --print-stats


Datum size: 1668


The choice has been recorded.

In [55]:
jq .state.choices guessing-game-07a.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 1.8483410299359473e+47
- - choice_name: Cipher Text
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 437428978988496300000000
- - choice_name: Plain Text is Positive
    choice_owner:
      pk_hash: bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f
  - 0
- - choice_name: Smaller Secret Prime
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 418133219637912500000000


Submit the transaction.

In [56]:
TX_7=$(
marlowe-cli run execute --marlowe-in-file guessing-game-06a.marlowe  \
                        --tx-in-marlowe "$TX_6#1"                    \
                        --tx-in-collateral "$TX_5#0"                 \
                        --tx-in "$TX_5#0"                            \
                        --required-signer $ALICE_SKEY                \
                        --change-address $ALICE_ADDR                 \
                        --marlowe-out-file guessing-game-07a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_7\""


Fee: Lovelace 1479606
Size: 16866 / 32768 = 51%
Execution units:
  Memory: 6678004 / 30000000 = 22%
  Steps: 2518384570 / 10000000000 = 25%
TxId "dd9b725672504e3d7cb5e1e86f7b8a7072ff293cf9996a20f79799f03fdeab55"


The Marlowe script has this UTxO at its address:

In [57]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p;/$TX_6/p;/$TX_7/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dd9b725672504e3d7cb5e1e86f7b8a7072ff293cf9996a20f79799f03fdeab55     1        202000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "34925397af60cd8560fd46895cc3161e838ddb0cc228110cb7edc19222e56d90"


Alice has these UTxOs at her address:

In [58]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ALICE_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p;/$TX_6/p;/$TX_7/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dd9b725672504e3d7cb5e1e86f7b8a7072ff293cf9996a20f79799f03fdeab55     0        783435280 lovelace + TxOutDatumNone


#### 8. Bob checks the secrets that Alice revealed and decides to challenge her

Bob checks that the smaller secret prime number is a divisor of the product that Alice reported.

In [59]:
echo "scale=30; $PRODUCT_OF_PRIMES / $SMALLER_PRIME" | bc

442045966004935111389041.000000000000000000000000000000


Bob makes sure the number that Alice reported as the smaller prime is actually a prime number:

In [60]:
factor $SMALLER_PRIME

418133219637912482167661: 418133219637912482167661


Bob makes sure that the number that Alice implied was the larger prime is actually a prime number.

In [61]:
factor $LARGER_PRIME

442045966004935111389041: 442045966004935111389041


Bob calculates the plain text, in order to see if he correctly guessed that it was negative.

In [62]:
echo "$CIPHER_TEXT - $SMALLER_PRIME" | bc

19295759350583799993723


Alice was telling the truth, so Bob does not challenges and waits until the game is over, when Alice receives her winnings.

In [63]:
marlowe-cli run prepare --invalid-before $END                      \
                        --invalid-hereafter $((END + 5 * MINUTES)) \
                        --marlowe-file guessing-game-07a.marlowe   \
                        --out-file     guessing-game-08a.marlowe   \
                        --print-stats


Datum size: 363
Payment 1
  Acccount: PK "bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f"
  Payee: Account (PK "c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f")
  Ada: 100.000000
Payment 2
  Acccount: PK "c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f"
  Payee: Party (PK "c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f")
  Ada: 202.000000


The plain text was correctly computed.

In [64]:
jq .state.boundValues guessing-game-08a.marlowe | json2yaml

- - Larger Secret Prime
  - 442045966004935100000000
- - Plain Text
  - 19295759350583800000000


The contract is complete.

In [65]:
jq .contract guessing-game-08a.marlowe | json2yaml

close
...


Wait unit the 30 minutes of the contract's duration has passed.

In [66]:
sleep $((END / 1000 - $(date -u +%s) + 120))s

Submit the transaction.

In [67]:
TX_8=$(
marlowe-cli run execute --marlowe-in-file guessing-game-07a.marlowe  \
                        --tx-in-marlowe "$TX_7#1"                    \
                        --tx-in-collateral "$TX_6#0"                 \
                        --tx-in "$TX_6#0"                            \
                        --required-signer $BOB_SKEY                  \
                        --change-address $BOB_ADDR                   \
                        --marlowe-out-file guessing-game-08a.marlowe \
                        --out-file /dev/null                         \
                        --submit 600                                 \
                        --print-stats                                \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_8\""


Fee: Lovelace 1219786
Size: 14551 / 32768 = 44%
Execution units:
  Memory: 4841130 / 30000000 = 16%
  Steps: 1797547297 / 10000000000 = 17%
TxId "640a6789a74119d3f23b05121304d4bdf71caefda6855942d67f067a2426067c"


The Marlowe script has no UTxOs at its address:

In [68]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p;/$TX_6/p;/$TX_7/p;/$TX_8/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Alice has these UTxOs at her address:

In [69]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ALICE_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p;/$TX_6/p;/$TX_7/p;/$TX_8/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
640a6789a74119d3f23b05121304d4bdf71caefda6855942d67f067a2426067c     1        202000000 lovelace + TxOutDatumNone
dd9b725672504e3d7cb5e1e86f7b8a7072ff293cf9996a20f79799f03fdeab55     0        783435280 lovelace + TxOutDatumNone


Bob has these UTxOs at his address:

In [70]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $BOB_ADDR | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p;/$TX_3/p;/$TX_4/p;/$TX_5/p;/$TX_6/p;/$TX_7/p;/$TX_8/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
640a6789a74119d3f23b05121304d4bdf71caefda6855942d67f067a2426067c     0        791599192 lovelace + TxOutDatumNone


### Alice Cheats

1.  Alice and Bob make their deposits.
2.  Alice secretly selects three primes 6240425729022497, 7445375476229597, and 9149696769535051.
3.  Alices chooses `Product of Secret Primes` = 425116072270803373983726731189192374944067744159.
4.  Alice cleverly chooses `Cipher Text` = 6984737363688812, which is between the two lower primes.
5.  Bob chooses `Plain Text is Positive`= 1, guessing that Alice's plain text is greater than zero.
6.  Since Alice sees that Bob guessed negative, Alice chooses `Smaller Secret Prime` = 7445375476229597, even though that is false, since this would mean that `Plain Text` = `Cipher Text` - `Smaller Secret Prime` would be negative and Bob would lose the wager.
7.  However, Bob runs the Linux command `factor 190125153026841891465807101762005016303854862683`, and in fifteen seconds discovers the three factors and sees that 6240425729022497 is lower than the one that Alice reported.
8.  Bob chooses `Smaller Challenge Prime` = 6240425729022497.
9.  The contract pays Bob because Bob proved that Alice cheated. If Bob had not challenged Alice, Alice's cheating would not have been detected and Alice would have won the wager.

#### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract.

In [71]:
marlowe-cli run initialize --contract-file guessing-game-01.contract  \
                           --state-file    guessing-game-01.state     \
                           --out-file      guessing-game-01b.marlowe  \
                           --print-stats


Validator size: 12582
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}


The minimum Ada deposit has been recorded.

In [72]:
jq .state guessing-game-01b.marlowe | json2yaml

accounts:
- - - pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1658677424000


We won't demonstrate running this example on the blockchain.

#### 2. Alice makes her deposit

In [73]:
marlowe-cli run prepare --deposit-account PK=$ALICE_PKH          \
                        --deposit-party   PK=$ALICE_PKH          \
                        --deposit-amount $WAGER                  \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-01b.marlowe \
                        --out-file     guessing-game-02b.marlowe \
                        --print-stats


Datum size: 2391


Now the contract still contains the initial Ada and the deposit.

In [74]:
jq .state guessing-game-02b.marlowe | json2yaml

accounts:
- - - pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
    - currency_symbol: ''
      token_name: ''
  - 102000000
boundValues: []
choices: []
minTime: 1658677424000


#### 3. Bob makes his deposit

In [75]:
marlowe-cli run prepare --deposit-account PK=$BOB_PKH            \
                        --deposit-party   PK=$BOB_PKH            \
                        --deposit-amount $WAGER                  \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-02b.marlowe \
                        --out-file     guessing-game-03b.marlowe \
                        --print-stats


Datum size: 2331


The contract's internal accounts have been updated.

In [76]:
jq .state.accounts guessing-game-03b.marlowe | json2yaml

- - - pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
    - currency_symbol: ''
      token_name: ''
  - 102000000
- - - pk_hash: bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f
    - currency_symbol: ''
      token_name: ''
  - 100000000


#### 4. Alice secretly selects prime numbers and chooses their product

Alice visits https://bigprimes.org/ and secretly selects the *three* primes 6240425729022497, 7445375476229597, and 9149696769535051.

In [77]:
PRIME_1=6240425729022497
PRIME_2=7445375476229597
PRIME_3=9149696769535051
PRODUCT_OF_PRIMES=$(echo "$PRIME_1 * $PRIME_2 * $PRIME_3" | bc)
echo $PRODUCT_OF_PRIMES

425116072270803373983726731189192374944067744159


In [78]:
marlowe-cli run prepare --choice-party PK=$ALICE_PKH             \
                        --choice-name "Product of Two Primes"    \
                        --choice-number $PRODUCT_OF_PRIMES       \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-03b.marlowe \
                        --out-file     guessing-game-04b.marlowe \
                        --print-stats


Datum size: 2276


The choice has been recorded.

In [79]:
jq .state.choices guessing-game-04b.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 4.2511607227080335e+47


👉 Note that `jq` rounds off the number when printing it, even though the full precision is present in the file `guessing-game-04b.marlowe`.

#### 5. Alice chooses the cipher text

Alice cleverly chooses Cipher Text = 6984737363688812, which is between the two lower primes.

In [80]:
CIPHER_TEXT=6984737363688812

In [81]:
marlowe-cli run prepare --choice-party PK=$ALICE_PKH             \
                        --choice-name "Cipher Text"              \
                        --choice-number $PLAIN_TEXT              \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-04b.marlowe \
                        --out-file     guessing-game-05b.marlowe \
                        --print-stats


Datum size: 2234


The choice has been recorded.

In [82]:
jq .state.choices guessing-game-05b.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 4.2511607227080335e+47
- - choice_name: Cipher Text
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 19295759350583800000000


#### 6. Bob guesses that the plain text is a positive number

Before guessing whether the plain text is a positive number or not, Bob tries to discover Alice's secret primes by trying to factor the large product that Alice reported.

In [83]:
timeout 10m factor $PRODUCT_OF_PRIMES

425116072270803373983726731189192374944067744159: 6240425729022497 7445375476229597 9149696769535051


Bob sees that Alices might be cheating and that Alice's cipher text is greater than the smallest prime factor, so he just guesses that Alice's plain text is a positive number.

In [84]:
marlowe-cli run prepare --choice-party PK=$BOB_PKH               \
                        --choice-name "Plain Text is Positive"   \
                        --choice-number 1                        \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-05b.marlowe \
                        --out-file     guessing-game-06b.marlowe \
                        --print-stats


Datum size: 2201


The choice has been recorded.

In [85]:
jq .state.choices guessing-game-06b.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 4.2511607227080335e+47
- - choice_name: Cipher Text
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 19295759350583800000000
- - choice_name: Plain Text is Positive
    choice_owner:
      pk_hash: bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f
  - 1


#### 7. Alice cheats by choosing a number that is not the smallest prime factor

Since Alice sees that Bob guessed negative, Alice chooses Smaller Secret Prime = 7445375476229597, even though that is false, since this would mean that `Plain Text = Cipher Text - Smaller Secret Prime` would be negative and Bob would lose the wager.

In [86]:
SMALLER_PRIME=$PRIME_2
echo $SMALLER_PRIME

7445375476229597


In [87]:
marlowe-cli run prepare --choice-party PK=$ALICE_PKH             \
                        --choice-name "Smaller Secret Prime"     \
                        --choice-number $SMALLER_PRIME           \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-06b.marlowe \
                        --out-file     guessing-game-07b.marlowe \
                        --print-stats


Datum size: 1669


The choice has been recorded.

In [88]:
jq .state.choices guessing-game-07b.marlowe | json2yaml

- - choice_name: Product of Two Primes
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 4.2511607227080335e+47
- - choice_name: Cipher Text
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 19295759350583800000000
- - choice_name: Plain Text is Positive
    choice_owner:
      pk_hash: bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f
  - 1
- - choice_name: Smaller Secret Prime
    choice_owner:
      pk_hash: c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f
  - 7445375476229597


#### 8. Bob checks the secrets that Alice revealed and decides to challenge her

Bob checks that the smaller secret prime number is a divisor of the product that Alice reported.

In [89]:
LARGER_PRIME=$(echo "$PRODUCT_OF_PRIMES / $SMALLER_PRIME" | bc)
echo $LARGER_PRIME

57098003133360556355951409042347


Bob makes sure the number that Alice reported as the smaller prime is actually a prime number:

In [90]:
factor $SMALLER_PRIME

7445375476229597: 7445375476229597


Bob makes sure that the number that Alice implied was the larger prime is actually a prime number, but sees that it is a composite number and that its smallest factor is smaller than the one that Alice reported as the smaller prime.

In [91]:
factor $LARGER_PRIME

57098003133360556355951409042347: 6240425729022497 9149696769535051


Bob calculates the plain text, in order to see if he correctly guessed that it was positive.

Bob chooses `Smaller Challenge Prime = 6240425729022497`.

In [92]:
CHALLENGE=6240425729022497

Alice was not telling the truth, so Bob challenges with a lower factor and he receives his winnings.

In [93]:
marlowe-cli run prepare --choice-party PK=$BOB_PKH               \
                        --choice-name "Smaller Challenge Prime"  \
                        --choice-number $CHALLENGE               \
                        --invalid-before $NOW                    \
                        --invalid-hereafter $LATER               \
                        --marlowe-file guessing-game-07b.marlowe \
                        --out-file     guessing-game-08b.marlowe \
                        --print-stats


Datum size: 451
Payment 1
  Acccount: PK "c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f"
  Payee: Account (PK "bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f")
  Ada: 100.000000
Payment 2
  Acccount: PK "c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f"
  Payee: Party (PK "c83ea613945cb8f6140a33341dadbd54803db63a037b0a3b31f8b14f")
  Ada: 2.000000
Payment 3
  Acccount: PK "bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f"
  Payee: Party (PK "bf58c039d16d33213fa640618ce5fbad94c73f85b7d0c982d935db6f")
  Ada: 200.000000


The contract is complete.

In [94]:
jq .contract guessing-game-08b.marlowe | json2yaml

close
...
